7. What is the average number of aces per match? 


In [ ]:
import pandas as pd
from glob import glob
import os


To create statistics_all.parquet:

In [2]:
import os
from glob import glob
import pandas as pd
from pathlib import Path

# Project paths
project_root = Path.cwd().parent
raw_tennis_dir = project_root / "data" / "raw" / "tennis_data"
out_path = project_root / "data" / "raw" / "statistics_all.parquet"

# Collect day folders
days_to_query = [d for d in os.listdir(raw_tennis_dir) if (raw_tennis_dir / d).is_dir()]

# Gather parquet file paths
statistics_load_files = []
for day in days_to_query:
    day_path = raw_tennis_dir / day / "data" / "raw" / "raw_statistics_parquet"
    statistics_load_files.extend(glob(str(day_path / "*.parquet")))

# Merge and save
if statistics_load_files:
    stat_df = pd.concat([pd.read_parquet(f) for f in statistics_load_files], ignore_index=True)
    stat_df.to_parquet(out_path, index=False)
    print(f"Saved full statistics dataset to {out_path} ({stat_df.shape[0]} rows).")
else:
    print("No statistics parquet files found.")


Saved full statistics dataset to c:\Users\mit\Desktop\Final_Project\data\raw\statistics_all.parquet (1358234 rows).


In [ ]:
aces_df = stat_df[stat_df["statistic_name"] == 'aces'].copy()

# Total aces count (sum over both teams)
aces_df['total_aces'] = aces_df['home_value'] + aces_df['away_value']
total_aces = aces_df['total_aces'].sum()
print(f"Total amount of aces: {total_aces}")

total_match = stat_df['match_id'].nunique()
print(f"Total amount of matches: {total_match}")

# Average aces per match
mean_aces = aces_df.groupby('match_id')['total_aces'].sum().mean()
print(f"Average number of aces per match: {mean_aces:.2f}")

out_dir = project_root / "reports" / "answers"
out_dir.mkdir(parents=True, exist_ok=True)
pd.DataFrame({
    "total_aces": [total_aces],
    "total_matches": [total_match],
    "average_aces_per_match": [mean_aces]
}).to_csv(out_dir / "q7_average_aces_per_match.csv", index=False, encoding="utf-8-sig")


Total amount of aces: 246656
Total amount of matches: 11393
Average number of aces per match: 21.66
